In [18]:
import pandas as pd

In [19]:

df = pd.read_hdf("./retention_features_new.h5")
print(df.columns)

Index(['user_id', 'card_id', 'card_text', 'is_new_fact',
       'user_n_study_positive', 'user_n_study_negative', 'user_n_study_total',
       'card_n_study_positive', 'card_n_study_negative', 'card_n_study_total',
       'usercard_n_study_positive', 'usercard_n_study_negative',
       'usercard_n_study_total', 'acc_user', 'acc_card', 'acc_usercard',
       'usercard_delta', 'usercard_delta_previous',
       'usercard_previous_study_response', 'leitner_box', 'sm2_efactor',
       'sm2_interval', 'sm2_repetition', 'delta_to_leitner_scheduled_date',
       'delta_to_sm2_scheduled_date', 'repetition_model',
       'elapsed_milliseconds', 'correct_on_first_try', 'utc_date', 'response',
       'n_minutes_spent', 'deck_id', 'deck_name'],
      dtype='object')


In [20]:
df.head

<bound method NDFrame.head of       user_id card_id                                          card_text  \
0          46   26811  People of the steppes of central Asia, governe...   
1          46   39163   Depose Hassan, the last Dey ruler of this city.    
2          46   31494  The historian Bede was from, this kingdom cent...   
3          46   22848  Kara Mustafa's attempts to take Vienna were th...   
4          46   33260  Man overthrown by Francisco Pizarro, an Incan ...   
...       ...     ...                                                ...   
99200     598   32264  The release of nitrogen gas from AIBN is coupl...   
99201     598   26600  This chemist is the first namesake of a reacti...   
99202     598   26630  Concerted substitution reaction mechanism whic...   
99203     598   20570  Gutmann introduced a measure that increases wi...   
99204     598   28907  These materials were discovered when it was no...   

       is_new_fact  user_n_study_positive  user_n_study_n

In [21]:
df_orig = df.copy()
# df.rename(columns = {'label':'response'}, inplace = True)

In [22]:
import uuid
# Adapted from 
# https://stackoverflow.com/questions/66366449/how-to-generate-8-digit-unique-identifier-to-replace-the-existing-one-in-python
original_ids = df['user_id'].unique()
new_ids = {cid: uuid.uuid4().int for cid in original_ids}
df['Anon Student Id'] = df['user_id'].map(new_ids)
# Make each session id a user id

In [23]:
 df['utc_date'].head

<bound method NDFrame.head of 0        2020-07-15
1        2020-07-15
2        2020-07-15
3        2020-07-15
4        2020-07-15
            ...    
99200    2020-12-18
99201    2020-12-18
99202    2020-12-18
99203    2020-12-18
99204    2020-12-18
Name: utc_date, Length: 99205, dtype: object>

In [24]:
# Note this gets rid of milliseconds, as it seems datashop only takes up to three decimal places while %f is six
df['Time'] = df['utc_date'].apply(lambda x: str(x) + " 00:00")
# df['Time'] = df['utc_date'].dt.strftime('%Y-%m-%d %H:%M:%S')
df['Time'].head

<bound method NDFrame.head of 0        2020-07-15 00:00
1        2020-07-15 00:00
2        2020-07-15 00:00
3        2020-07-15 00:00
4        2020-07-15 00:00
               ...       
99200    2020-12-18 00:00
99201    2020-12-18 00:00
99202    2020-12-18 00:00
99203    2020-12-18 00:00
99204    2020-12-18 00:00
Name: Time, Length: 99205, dtype: object>

In [25]:
# No need for level? Is level different from KC
# https://pslcdatashop.web.cmu.edu/help?page=importFormatTd
df['Level (Deck)'] = df['deck_id'].apply(lambda x:"Level (" + x + ")")
# df['Problem Name'] = df['card_text']

In [26]:
df['Problem Name'] = df['card_id']

In [27]:
# df['Action'] = "Answer Question" 

In [28]:
df['Condition Name'] = df['repetition_model']

In [29]:
df['Condition Type'] = "Repetition Model"

In [31]:
# df.drop(['user_id'], inplace=True, axis=1)
df["Outcome"] = df['response'].apply(lambda x: "CORRECT" if x else "INCORRECT")
# df['Step Name'] = df['usercard_n_study_total']
df['Step Name'] = "Flashcard"
# df['Session ID'] = df['index']
learnsphere_df = df[["Anon Student Id", "Time", "Level (Deck)", "Problem Name", "Step Name", "Outcome", "Condition Name", "Condition Type"]]
# TODO: Create session id

In [ ]:
learnsphere_df.index.names = ["Session Id"]

In [ ]:
learnsphere_df.to_csv("22-07-27-learnsphere_karl.txt", sep="\t")